**808. Soup Servings**

**Medium**

**Companies : Google**

You have two soups, A and B, each starting with n mL. On every turn, one of the following four serving operations is chosen at random, each with probability 0.25 independent of all previous turns:

- pour 100 mL from type A and 0 mL from type B
- pour 75 mL from type A and 25 mL from type B
- pour 50 mL from type A and 50 mL from type B
- pour 25 mL from type A and 75 mL from type B

  > Note:

- There is no operation that pours 0 mL from A and 100 mL from B.
- The amounts from A and B are poured simultaneously during the turn.
- If an operation asks you to pour more than you have left of a soup, pour all that remains of that soup.

The process stops immediately after any turn in which one of the soups is used up.

Return the probability that A is used up before B, plus half the probability that both soups are used up in the same turn. Answers within 10-5 of the actual answer will be accepted.

**Example 1:**

```python

Input: n = 50
Output: 0.62500
```

**Explanation:**

- If we perform either of the first two serving operations, soup A will become empty first.

- If we perform the third operation, A and B will become empty at the same time.

- If we perform the fourth operation, B will become empty first.

- So the total probability of A becoming empty first plus half the probability that A and B become empty at the same time, is 0.25 \* (1 + 1 + 0.5 + 0) = 0.625.

**Example 2:**

```python

Input: n = 100
Output: 0.71875
```

**Explanation**:

- If we perform the first serving operation, soup A will become empty first.

- If we perform the second serving operations, A will become empty on performing operation [1, 2, 3], and both A and B become empty on performing operation 4.

- If we perform the third operation, A will become empty on performing operation [1, 2], and both A and B become empty on performing operation 3.

- If we perform the fourth operation, A will become empty on performing operation 1, and both A and B become empty on performing operation 2.

- So the total probability of A becoming empty first plus half the probability that A and B become empty at the same time, is 0.71875.

**Constraints**:

- 0 <= n <= 109


In [ ]:
# The algorithm combines a dynamic programming approach with a convergence optimization.
# For large values of n, the probability is very close to 1, so we can return 1.
# For smaller values, we use memoization to compute the probability recursively.

class Solution:
    def soupServings(self, n: int) -> float:
        # Convert n to units of 25 mL and take the ceiling.
        # This simplifies the problem by working with integers.
        # ceil(n/25) is equivalent to (n + 24) // 25
        n_units = (n + 24) // 25

        # As n becomes large, the probability converges to 1.
        # We can find a threshold where the difference from 1 is less than 10^-5.
        # This threshold is approximately 4800.
        if n_units >= 192:  # 192 * 25 = 4800
            return 1.0

        # Memoization dictionary to store the results of subproblems.
        # Keys are tuples (a, b), values are the calculated probabilities.
        memo = {}

        def solve(a, b):
            # Base cases for the recursion.
            # a and b are the number of 25 mL units remaining.
            
            # If both soups are empty, the probability is 0.5 (for simultaneous depletion).
            if a <= 0 and b <= 0:
                return 0.5
            # If only soup A is empty, the probability is 1.0.
            if a <= 0:
                return 1.0
            # If only soup B is empty, the probability is 0.0.
            if b <= 0:
                return 0.0

            # Check if we have already calculated this state.
            if (a, b) in memo:
                return memo[(a, b)]

            # Recursive step: calculate the probability based on the four operations.
            # Each operation has a probability of 0.25.
            # The amounts are in units of 25 mL, so the operations are:
            # 1. (4, 0) -> a-4, b
            # 2. (3, 1) -> a-3, b-1
            # 3. (2, 2) -> a-2, b-2
            # 4. (1, 3) -> a-1, b-3
            
            result = 0.25 * (
                solve(a - 4, b) +
                solve(a - 3, b - 1) +
                solve(a - 2, b - 2) +
                solve(a - 1, b - 3)
            )

            # Store the result in the memoization table before returning.
            memo[(a, b)] = result
            return result

        # Start the recursion with the initial amounts of soup.
        return solve(n_units, n_units)

In [ ]:
# The bottom-up approach uses a 2D array to build up the solution from base cases.
# This avoids recursion depth limits and can be more memory efficient.

class Solution:
    def soupServings(self, n: int) -> float:
        n_units = (n + 24) // 25

        if n_units >= 192:
            return 1.0

        # DP table to store probabilities.
        # dp[i][j] will be the probability for i units of A and j units of B.
        dp = [[0.0] * (n_units + 1) for _ in range(n_units + 1)]

        # Iterate through the DP table, filling it with the calculated probabilities.
        for i in range(n_units + 1):
            for j in range(n_units + 1):
                # Base case: if both are 0, probability is 0.5.
                if i == 0 and j == 0:
                    dp[i][j] = 0.5
                # Base case: if only A is 0, probability is 1.
                elif i == 0:
                    dp[i][j] = 1.0
                # Base case: if only B is 0, probability is 0.
                elif j == 0:
                    dp[i][j] = 0.0
                # Recursive step from the perspective of tabulation.
                # We need to consider the four possible previous states that lead to (i, j).
                else:
                    dp[i][j] = 0.25 * (
                        dp[max(0, i - 4)][j] +
                        dp[max(0, i - 3)][max(0, j - 1)] +
                        dp[max(0, i - 2)][max(0, j - 2)] +
                        dp[max(0, i - 1)][max(0, j - 3)]
                    )

        # The final answer is at dp[n_units][n_units].
        return dp[n_units][n_units]

In [ ]:
# The solution uses a top-down dynamic programming approach with memoization.
# It's based on a recursive function that explores all possible serving operations.
# The core idea is to find the probability for each of the four possible next states
# and sum them up, multiplying by 0.25 (since each operation has an equal probability).

class Solution:
    def soupServings(self, n: int) -> float:
        # A trick for large 'n':
        # As 'n' gets very large, the probability that soup A runs out first
        # approaches 1.0. This is because soup A is consumed, on average, faster
        # than soup B. The problem statement's required precision is 10^-5,
        # and for n >= 4800, the probability is already > 0.99999.
        # Returning 1.0 for large 'n' saves significant computation time.
        # The C++ code uses 5000, which is also a safe threshold.
        if n >= 4800:
            return 1.0

        # The memoization table, `t`, stores results for subproblems.
        # `t[i][j]` will hold the probability for 'i' ml of soup A and 'j' ml of soup B.
        self.t = {}

        def solve(a, b):
            # Base Case 1: Both soups A and B are empty simultaneously.
            # The problem asks for half this probability, which is 0.5.
            if a <= 0 and b <= 0:
                return 0.5
            
            # Base Case 2: Only soup A is empty.
            # This is a favorable outcome, so the probability is 1.0.
            if a <= 0:
                return 1.0
            
            # Base Case 3: Only soup B is empty.
            # This is an unfavorable outcome, so the probability is 0.0.
            if b <= 0:
                return 0.0
            
            # Check memoization table: If we've already solved this subproblem,
            # return the stored result to avoid recomputing.
            if (a, b) in self.t:
                return self.t[(a, b)]

            # Recursive step: Calculate the probability by considering all four serving operations.
            # Each operation has a probability of 0.25.
            # We add up the probabilities of the four subsequent states.
            
            # Operation 1: Pour 100 mL from A, 0 mL from B
            prob1 = 0.25 * solve(a - 100, b)
            
            # Operation 2: Pour 75 mL from A, 25 mL from B
            prob2 = 0.25 * solve(a - 75, b - 25)
            
            # Operation 3: Pour 50 mL from A, 50 mL from B
            prob3 = 0.25 * solve(a - 50, b - 50)
            
            # Operation 4: Pour 25 mL from A, 75 mL from B
            prob4 = 0.25 * solve(a - 25, b - 75)
            
            total_probability = prob1 + prob2 + prob3 + prob4
            
            # Store the computed result in the memoization table before returning.
            self.t[(a, b)] = total_probability
            return total_probability
        
        # Start the recursion with the initial amount of soup 'n' for both A and B.
        return solve(n, n)
solution = Solution()

# Example 1: n = 50
# Expected Output: 0.625
print(f"n = 50 -> Probability: {solution.soupServings(50)}")

# Example 2: n = 100
# Expected Output: 0.71875
print(f"n = 100 -> Probability: {solution.soupServings(100)}")

# Edge Case 1: n = 0
# Both soups start empty, so they are used up simultaneously. The probability is 0.5.
# Expected Output: 0.5
print(f"n = 0 -> Probability: {solution.soupServings(0)}")

# Edge Case 2: n = 25
# There are four possible outcomes, each with a probability of 0.25:
# - (100, 0): A becomes 0 first. P = 1
# - (75, 25): A becomes 0 first. P = 1
# - (50, 50): A and B become 0 simultaneously. P = 0.5
# - (25, 75): B becomes 0 first. P = 0
# Total probability = 0.25 * (1 + 1 + 0.5 + 0) = 0.625
# Expected Output: 0.625
print(f"n = 25 -> Probability: {solution.soupServings(25)}")

# Test Case for Large 'n'
# For n >= 4800, the probability converges to 1.0.
# Expected Output: 1.0
print(f"n = 5000 -> Probability: {solution.soupServings(5000)}")